In [1]:
import os

In [2]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Shreyansh-1998/MLOps_CNNClassifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Shreyansh-1998"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "@@@@@@@@"

In [6]:
import tensorflow as tf


In [7]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int


In [9]:
from MLOPSCNNCLASSIFIER.constants import *
from MLOPSCNNCLASSIFIER.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

             self.config = read_yaml(config_filepath)
             self.params = read_yaml(params_filepath)
             create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self):
        eval_config=EvaluationConfig(
               path_of_model="artifacts/training/model.h5",
               training_data="artifacts/data_ingestion/KIDNEY_CT_SCAN",
               mlflow_uri="https://dagshub.com/Shreyansh-1998/MLOps_CNNClassifier.mlflow",
               all_params=self.params,
               params_image_size=self.params.IMAGE_SIZE,
               params_batch_size=self.params.BATCH_SIZE

           )
        return eval_config 


In [11]:
pip install mlflow==2.5.0

Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse



In [13]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):
        
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split=0.30
        )

        dataflow_kwargs= dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=  self.config.training_data,
            subset  ="validation",
            shuffle= False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path):
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    #mlflow integration
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_registry_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],
                 "accuracy":self.score[1]}
                 )
            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")

            else:
                mlflow.keras.log_model(self.model,"model")



In [14]:
try:
    config_manager=ConfigurationManager()
    eval_config=config_manager.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()    
    
except Exception as e:
    raise e

[2024-03-22 15:59:13,064: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-03-22 15:59:13,067: INFO: common: YAML file: params.yaml loaded successfully]
[2024-03-22 15:59:13,068: INFO: common: Directory created at artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 264s 2s/step - loss: 2.8497 - accuracy: 0.7417
[2024-03-22 16:03:37,128: INFO: common: json file saved at scores.json]


2024/03/22 16:03:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/l2/8f2r1rwx6lx76vv6n6ncb1k40000gn/T/tmpbv606wje/model/data/model/assets
[2024-03-22 16:03:38,992: INFO: builder_impl: Assets written to: /var/folders/l2/8f2r1rwx6lx76vv6n6ncb1k40000gn/T/tmpbv606wje/model/data/model/assets]


/opt/anaconda3/envs/venv/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/03/22 16:03:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
